In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image

class Meso4(nn.Module):
    def __init__(self):
        super(Meso4, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(8, 16, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(16, 16, 5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 8 * 8, 16)  
        self.fc2 = nn.Linear(16, 2)  
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.softmax(x)

class DeepFakeDataset(Dataset):
    def __init__(self, real_dir, fake_dir, transform=None):
        self.real_images = [os.path.join(real_dir, img) for img in os.listdir(real_dir) if img.endswith(".jpg")]
        self.fake_images = [os.path.join(fake_dir, img) for img in os.listdir(fake_dir) if img.endswith(".jpg")]
        self.transform = transform
        self.data = [(img, 0) for img in self.real_images] + [(img, 1) for img in self.fake_images]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_real_dir = r"C:\Users\rites\OneDrive\Desktop\archive.zip\AI-face-detection-Dataset\real"
train_fake_dir = r"C:\Users\rites\OneDrive\Desktop\archive.zip\AI-face-detection-Dataset\AI"
val_real_dir = r"C:\Users\rites\OneDrive\Desktop\archive (1).zip\test-20250112T065939Z-001\test\real"
val_fake_dir = r"C:\Users\rites\OneDrive\Desktop\archive (1).zip\test-20250112T065939Z-001\test\fake"

train_dataset = DeepFakeDataset(train_real_dir, train_fake_dir, transform=transform)
val_dataset = DeepFakeDataset(val_real_dir, val_fake_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

def resume_training(model, train_loader, val_loader, epochs=15, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Load Meso2V1
    checkpoint_path = "deepfake_model.pth"
    if os.path.exists(checkpoint_path):
        print("Loading saved model...")
        model.load_state_dict(torch.load(checkpoint_path))  # Load model weights
        print("Model Meso2V1 loaded successfully!")

    best_val_acc = 0.0  # validation accuracy
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_acc = 100 * correct_train / total_train

        # validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

        val_acc = 100 * correct_val / total_val
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

        # Meso2V1_1.13 created
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "Meso2V1_1.13(new_image).pth")
            print("Model Meso2V1_1.13 saved")

    print("Training complete!")

model = Meso4()
resume_training(model, train_loader, val_loader, epochs=10)


ModuleNotFoundError: No module named 'torch'